In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
%%javascript
var nb = IPython.notebook;
var kernel = IPython.notebook.kernel;
var command = "nb_path = '" + nb.base_url + nb.notebook_path + "'";
kernel.execute(command);

<IPython.core.display.Javascript object>

# Calculate limb darkening coefficients

First... some imports!

In [26]:
import os, sys
dirname = os.path.dirname(nb_path)
sys.path.append(dirname)

from ExoCTK.ExoCTK import core
from ExoCTK.ExoCTK.limb_dark import ldcfit as lf
from ExoCTK.ExoCTK.limb_dark import ldcplot as lp
import matplotlib.pyplot as plt

To calculate the limb darkening coefficients, we need a model grid. 

I'm going to use the Phoenix ACES models for this example but any grid can be loaded into a `core.ModelGrid()` object if the spectra are stored as FITS files.

We will pass a directory or directory+wildcard path to the initialize the model grid.

In [ ]:
fits_files = '../../../limb_dark_jeff/limb/specint/lte*.PHOENIX-ACES-AGSS-COND-SPECINT-2011.fits'
model_grid = core.ModelGrid(fits_files)
print(model_grid.data)

Now let's customize it to our desired effective temperature, surface gravity, metallicity, and wavelength ranges by running the `customize()` method on our grid.

In [ ]:
model_grid.customize(Teff_rng=(2500,2600), logg_rng=(5,5.5), FeH_rng=(0,0), wave_rng=(1.4,1.9))

Now we can caluclate the limb darkening coefficients using the `ldcfit.ldc()` function.

We just need to specify the desired effective temperature, surface gravity, metallicity, and the function to fit to the limb darkening profile (including 'uniform', 'linear', 'quadratic', 'square-root', 'logarithmic', 'exponential', and 'nonlinear').

We can do this with for a single model on the grid...

In [ ]:
teff, logg, FeH = 2500, 5.0, 0.0
coeffs, mu, radius = lf.ldc(teff, logg, FeH, model_grid, 'nonlinear', plot=True)

...or a single model off the grid...

In [ ]:
teff, logg, FeH = 2532, 5.1, 0.0
coeffs, mu, radius = lf.ldc(teff, logg, FeH, model_grid, 'nonlinear', plot=True)

...or the entire grid using the `ldcfit.ldc_grid()` function!

In [ ]:
coeff_grid, mu_grid, radius_grid = lf.ldc_grid(model_grid, 'nonlinear', write_to='', plot=True)

Alternatively, the results can be written to a FITS file with COEFFS, MU, and RADII extensions by specifying a filepath with the `write_to` argument.

## Using a Photometric Bandpass

Above we caluclated the limb darkening in a particular wavelength range set when we ran the `customize()` method on our `core.ModelGrid()` object. 

Additionally, we can calculate the limb darkening through a particular photometric bandpass.

First we have to create a `core.Filter()` object which we can then pass to the functions in the `ldcfit` module. Let's use 2MASS H-band for this example.

In [ ]:
H_band = core.Filter('2MASS.H')
plt.plot(*H_band.rsr)

Now we can tell `ldcfit.ldc()` to apply the filter to the models before calculating the limb darkening coefficients using the `bandpass` argument. We'll compare the results of using the bandpass (blue line) to the plot above where we just used the wavelength window of 1.4-1.9 $\mu m$ (green line).

In [ ]:
teff, logg, FeH = 2500, 5.0, 0.0
coeffs, mu, radius = lf.ldc(teff, logg, FeH, model_grid, 'nonlinear', bandpass=H_band, plot=True)
coeffs, mu, radius = lf.ldc(teff, logg, FeH, model_grid, 'nonlinear', plot=plt.gcf())

Enjoy!